In [1]:
import torch.nn.functional as F
import torch
from joblib import dump, load
import torch.utils.data as Data
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

class SelfAttention(nn.Module):
    def __init__(self, embed_dim):
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        attention_weights = F.softmax(torch.matmul(q, k.transpose(1, 2)) / torch.sqrt(torch.tensor(self.embed_dim)), dim=-1)
        output = torch.matmul(attention_weights, v)
        # print("SelfAttention shape:"+str(output.shape))
        return output

In [2]:
import torch
# 定义 EALSETM 模型
class EALSETM(nn.Module):
    def __init__(self, batch_size, input_dim, hidden_layer_sizes, output_dim, attention_dim):
        """
        预测任务  params:
         batch_size       : 批次量大小
        input_dim         : 输入数据的维度
        hidden_layer_size : 隐层的数目和维度
        output_dim        : 输出维度
        attention_dim     : 自注意力机制维度

        """
        super().__init__()
        # 批次量大小
        self.batch_size = batch_size
        # lstm层数
        self.num_layers = len(hidden_layer_sizes)
        self.hidden_layer_sizes = len(hidden_layer_sizes)
        self.lstm_layers = nn.ModuleList()  # 用于保存LSTM层的列表

        # 定义第一层LSTM
        self.lstm_layers.append(nn.LSTM(input_dim, hidden_layer_sizes[0], batch_first=True))

        # 定义后续的LSTM层
        for i in range(1, self.num_layers):
                self.lstm_layers.append(nn.LSTM(hidden_layer_sizes[i-1], hidden_layer_sizes[i], batch_first=True))

        # 定义自注意力层
        self.attention = SelfAttention(attention_dim)

         # 序列平均池化操作
        self.adaptive_pool = nn.AdaptiveAvgPool1d(1)


        # 定义线性层
        self.linear  = nn.Linear(hidden_layer_sizes[-1], output_dim)


    def forward(self, input_seq):  # torch.Size([16, 512])
        # 前向传播的过程是输入->LSTM层->全连接层->输出
        # 在观察查看LSTM输入的维度，LSTM的第一个输入input_size维度是(batch, seq_length, dim) batch是batch size , seq_length是序列长度，dim是输入维度，也就是变量个数
        # LSTM的第二个输入是一个元组，包含了h0,c0两个元素，这两个元素的维度都是（D∗num_layers,N,out)，
        # D=1表示单向网络，num_layers表示多少个LSTM层叠加，N是batch size，out表示隐层神经元个数

        # 数据预处理
        #改变输入形状，适应网络输入[batch, seq_length, dim]
        lstm_out = input_seq
        # 获取隐藏层数据
        hidden_states = []
        for lstm in self.lstm_layers:
            lstm_out, hidden= lstm(lstm_out)  ## 进行一次LSTM层的前向传播
            hidden_states += hidden
        # print("lstm_out.size"+str(lstm_out.size()))  # torch.Size([64, 7, 64])

        # 送入注意力机制层
        attention_features = self.attention(lstm_out)  # torch.Size([64, 7, 64])

         # 自适应平均池化
        x = self.adaptive_pool(attention_features.permute(0,2,1))  # torch.Size([64, 64, 1])
        # 平铺
        x = x.view(self.batch_size, -1)  # 把数据平铺
        # 全连接层
        predict = self.linear(x) # torch.Size([64, 10]
        return predict


state = torch.load("final_model_lstm_att.pt")
# weight =state['ModuleList.weight']
print(state['state_dict'].keys())
print("权重",state)

TypeError: 'EALSETM' object is not subscriptable